In [4]:
import cv2
import logging
import numpy
import typing
from utils import (
    create_folder, 
    get_logger, 
    debounce, 
    throttle, 
    get_homography_martix, 
    select_points
)
from part1_interface import(
    gaussian_2d_convolution,
    convolution_2D,
)
logger: logging.Logger = get_logger(__name__)

In [5]:
image1 = cv2.imread('im011/im01.jpg')
image1 = cv2.cvtColor(src=image1, code=cv2.COLOR_BGR2GRAY)
image2 = cv2.imread('im011/im02.jpg')
image2 = cv2.cvtColor(src=image2, code=cv2.COLOR_BGR2GRAY)
image2 = cv2.resize(src=image2, dsize=(image1.shape[1], image1.shape[0]))

In [6]:
def get_key_points():
    key_points = []
    pass
def get_key_points_decipher(image: numpy.ndarray, 
                            num_features:int = 6, 
                            sigma: float=1.52,

)-> typing.Tuple[typing.List[typing.Any], 
                 numpy.ndarray[typing.Any, 
                               typing.Any]]:
    """
    :param image: input image
    :param num_features: the num of features to extract
    :param sigma: sigma of the first stack.
                  it's better to use √(1.6^2 - 0.5^2) which is the defualt value
    :
    """
    scale_factor = 1
    s = num_features  + 3
    o = max(int(numpy.log2(min(image.shape[0], image.shape[1]))) - 3, 3)
    k = 2 **(1/num_features)
    sigmas: typing.List[typing.List[float]] = []
    origin_images: typing.List[numpy.ndarray] = []
    resized_image = image
    for O in range(o):
        row = []
        for S in range(s):
            value = (k ** S) * sigma * (1 << O)
            row.append(value)
        sigmas.append(row)
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)
        scale_factor /= 2
        resized_image = cv2.resize(resized_image, (new_width, new_height))
        origin_images.append(resized_image)
    guassian_pyramid: typing.List[typing.List[numpy.ndarray]] = []
    for i in range(0):
        current_gussian: typing.List[numpy.ndarray]= []
        guassian_pyramid.append(current_gussian)
        for j in range(s):
            cur_sigma:float = sigmas[i][j]
            size:int = int(cur_sigma * 6 + 1)
            guassian_2d_kernel = gaussian_2d_convolution(size=size, sigma=cur_sigma)
            guassian_result = convolution_2D(src=origin_images[i], kernel=guassian_2d_kernel)
            current_gussian.append(guassian_result)
    DoG: typing.List[typing.List[numpy.ndarray]] = []
    for O in range(o):
        DoG_row:typing.List[numpy.ndarray] = []
        DoG.append(DoG_row)
        for S in range(s - 1):
            diff_of_gaussian = guassian_pyramid[o][s + 1] - guassian_pyramid[o][s]
            DoG_row.append(diff_of_gaussian)
    key_points = []
    for O in range(0):
        for S in range(1, s-1):
            pass
    
    return (1,2);
    pass